In [1]:
import glob
import io
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
import os
import openslide
from openslide import open_slide
import pandas as pd
from PIL import Image, ImageOps
import seaborn as sns
import SimpleITK as sitk
from skimage.color import rgb2hed
from skimage.exposure import histogram
from skimage.filters import threshold_otsu
from skimage import morphology
import time
import warnings

In [2]:
SLIDES_PATH = '/90days/s4596423/Data'

In [3]:
os.chdir(SLIDES_PATH)
os.getcwd()

'/gpfs1/scratch/90days/s4596423/Data'

In [4]:
slides = []
for infile in glob.glob('*.svs'):
    file, ext = os.path.splitext(infile)
    slides.append(infile)
slides.sort()
slides


['1820_N_10545A_2_HandE.svs',
 '1820_N_10545A_4_TP53.svs',
 '2148_N_11397A_2_HandE.svs',
 '2148_N_11397A_4_TP53.svs',
 '2148_T_11393A_2_HandE.svs',
 '2148_T_11393A_4_TP53.svs',
 '2171_N_11521A_2_HandE.svs',
 '2171_N_11521A_4_TP53.svs',
 '2171_T_11524A_2_HandE.svs',
 '2171_T_11524A_4_TP53.svs']

In [5]:
tp53_slide = open_slide('2171_T_11524A_4_TP53.svs')
he_slide = open_slide('2171_T_11524A_2_HandE.svs')

In [6]:
def highest_mag(slide):
    """Returns the highest magnification for the slide
    """
    return int(slide.properties['aperio.AppMag'])

def level_mags(slide):
    """Returns the magnification for each level in a slide
    """
    return [highest_mag(slide)/downsample for downsample in slide.level_downsamples]

def get_level_size(slide, level):
    """Returns the dimensions of a level
    """
    return slide.level_dimensions[level]

def get_level_mag(slide, level):
    """Returns the magnification at a particular level
    """
    return level_mags(slide)[level]

def get_level_for_mag(slide, mag):
    """Get the level corresponding to a certain magnification, if available
    """
    level_mags_rounded = list(np.round(level_mags(slide), decimals = 2))
    if mag in level_mags_rounded:
        return level_mags_rounded.index(mag)
    else: 
        return None
    
def get_mag_for_size(slide, size):
    max_size = slide.dimensions
    max_mag = highest_mag(slide)
    downsample = np.average([max_dim/size_dim for max_dim, size_dim in zip(max_size, size)])
    return max_mag/downsample

def get_size_for_mag(slide, mag):
    max_size = slide.dimensions
    max_mag = highest_mag(slide)
    downsample = max_mag/mag
    return [np.int(np.round(dim/downsample)) for dim in max_size]

def read_slide_at_mag(slide, mag):
    exact_level = get_level_for_mag(slide, mag)
    if exact_level is not None:
        return slide.read_region((0,0), exact_level, get_level_size(slide, exact_level))
    else:
        max_size = slide.dimensions
        region_size = tuple(get_size_for_mag(slide, mag))
        downsample = np.average([max_dim/region_dim for max_dim, region_dim in zip(max_size, region_size)])
        best_level = slide.get_best_level_for_downsample(downsample)
        best_level_size = get_level_size(slide, best_level)
        best_level_img = slide.read_region((0,0), best_level, best_level_size)
        return best_level_img.resize(region_size, resample = Image.BICUBIC)   

In [9]:
#############################
# Image Filtering Functions #
#############################

def filter_green_channel(img, g_thresh = 240):
    new_img = img.copy()
    pixels = new_img.load()
    for i in range(img.size[0]):   #For every column
        for j in range(img.size[1]):    #For every row
            if pixels[i,j][2] > g_thresh:
                pixels[i,j] = (255,255,255)
    return new_img        

def filter_green_fast(img, g_thresh = 240):
    #About 18x faster than previous implementation
    img = img.convert('RGB')
    r, g, b = img.split()
    green_mask = (np.array(g) > 240)*255
    green_mask_img = Image.fromarray(green_mask.astype(np.uint8), 'L')
    white_image = Image.new('RGB', img.size, (255,255,255))
    img_filtered = img.copy()
    img_filtered.paste(white_image, mask = green_mask_img)
    return img_filtered

def filter_grays(img, tolerance = 3):
    new_img = img.convert('RGB')
    pixels = new_img.load()
    for i in range(img.size[0]):
        for j in range(img.size[1]):
            r, g, b = pixels[i,j]
            rg_diff, rb_diff, gb_diff = abs(r-g) <= tolerance, abs(r-b) <= tolerance, abs(g-b) <= tolerance
            if rg_diff and rb_diff and gb_diff:
                pixels[i,j] = (255,255,255)
    return new_img

def filter_otsu_global(img, mode = 'PIL'):
    img_gray = img.convert('L')
    threshold = threshold_otsu(np.array(img_gray))
    img_binary = np.array(img_gray) > threshold
    if mode == '1':
        return img_binary
    else:
        return binary_array_to_pil(img_binary)

def binary_array_to_pil(array):
    img_shape = array.shape
    img = Image.new('1', img_shape)
    int_list = array.astype(int).tolist()
    pixels = img.load()
    for i in range(img.size[0]):
        for j in range(img.size[1]):
            pixels[i,j] = int_list[i][j]
    return ImageOps.mirror(img).rotate(90, expand = True) #Not sure why this is necessary

def white_pixels_to_black(img):
    new_img = img.convert('RGB')
    pixels = new_img.load()
    for i in range(img.size[0]):
        for j in range(img.size[1]):
            rgb = pixels[i,j]
            if rgb == (255,255,255):
                pixels[i,j] = (0,0,0)
    return new_img

def tile_gen(img, tile_size):
    '''Generates tiles for Pillow images
    '''
    width, height = img.size
    x_tiles = int(np.floor(width/tile_size))
    y_tiles = int(np.floor(height/tile_size))
    yield (x_tiles, y_tiles)
    for y in range(y_tiles):
        for x in range(x_tiles):
            x_coord = x*tile_size
            y_coord = y*tile_size
            yield img.crop((x_coord, y_coord, np.int(np.round(x_coord+tile_size)), np.int(np.round(y_coord+tile_size))))

In [19]:
tuple(get_size_for_mag(tp53_slide, 10))
SLIDE_MAG = 5
he = read_slide_at_mag(he_slide, SLIDE_MAG)
tp53 = read_slide_at_mag(tp53_slide, SLIDE_MAG)

In [20]:
def get_itk_from_pil(pil_img):
    return sitk.GetImageFromArray(np.array(pil_img))

def get_pil_from_itk(itk_img):
    return Image.fromarray(sitk.GetArrayFromImage(itk_img).astype(np.uint8))

def show_alignment_gray(fixed_img, moving_img, resize = None):
    blank_img = Image.new(('L'), fixed_img.size, color = 0)
    comparison = Image.merge('RGB', [moving_img, fixed_img, blank_img])
    if resize is None:
        return comparison
    else: 
        downsample = max(comparison.size)/resize
        final_size = tuple([np.int(np.round(dim/downsample)) for dim in comparison.size])
        return comparison.resize(final_size, resample = Image.BICUBIC)

In [22]:
def elastix_params_for_mag(mag):
    a_param_map = sitk.GetDefaultParameterMap('affine')
    a_param_map['MaximumNumberOfIterations'] = [str(mag*512)]
    #a_param_map['NumberOfSpatialSamples'] = [str(mag**2*2048)]
    a_param_map['NumberOfSpatialSamples'] = ['2048']
    print('Number of Affine Spatial Samples: ', a_param_map['NumberOfSpatialSamples'])
    b_param_map = sitk.GetDefaultParameterMap('bspline')
    b_param_map['NumberOfResolutions'] = ['5']
    b_param_map['GridSpacingSchedule'] = ['6.6','4.0','2.6', '1.6', '1.0']
    b_param_map['Transform'] = ['RecursiveBSplineTransform']
    b_param_map['MaximumNumberOfIterations'] = [str(mag*1024)]
    print('Max BSpline Iterations: ', b_param_map['MaximumNumberOfIterations'])
    #b_param_map['NumberOfSpatialSamples'] = [str(mag**2*4096)]
    b_param_map['NumberOfSpatialSamples'] = ['4096']
    print('Number of BSpline Spatial Samples: ', b_param_map['NumberOfSpatialSamples'])
    #b_param_map['FinalGridSpacingInPhysicalUnits'] = [str((mag)*20)]
    b_param_map['FinalGridSpacingInPhysicalUnits'] = ['1080','540', '180', '60', '20']
    print('Final Grid Spacing in Physical Units: ', b_param_map['FinalGridSpacingInPhysicalUnits'])
    b_param_map['Metric'] = ['NormalizedMutualInformation']   #Faster with unnormalised version
    param_map_vector = sitk.VectorOfParameterMap()
    param_map_vector.append(a_param_map)
    param_map_vector.append(b_param_map)
    return param_map_vector

In [23]:
tp53_gray = tp53.convert('L')
he_gray = he.convert('L')
#---------------
tp53_itk = get_itk_from_pil(tp53_gray)
he_itk = get_itk_from_pil(he_gray)
#---------------
fixed_img = he_itk
moving_img = tp53_itk
param_map = elastix_params_for_mag(SLIDE_MAG)

Number of Affine Spatial Samples:  ('2048',)
Max BSpline Iterations:  ('5120',)
Number of BSpline Spatial Samples:  ('4096',)
Final Grid Spacing in Physical Units:  ('1080', '540', '180', '60', '20')


In [ ]:
start = time.time()

elastix = sitk.ElastixImageFilter()
elastix.SetFixedImage(fixed_img)
elastix.SetMovingImage(moving_img)
elastix.LogToFileOn()

elastix.SetParameterMap(param_map)
#elastix.LogToConsoleOn()
elastix.Execute()

result_img = elastix.GetResultImage()
transform_param_map = elastix.GetTransformParameterMap()

end = time.time()
print(end - start)

In [15]:
start = time.time()
tp53_filtered = filter_green_fast(tp53_aligned)
he_filtered = filter_green_fast(he)
green_end = time.time()
tp53_filtered = filter_grays(tp53_filtered, tolerance = 3)
he_filtered = filter_grays(he, tolerance = 15)
gray_end = time.time()
print('Time to Filter Green: {0}'.format(green_end - start))
print('Time to Filter Gray: {0}'.format(gray_end - green_end))

NameError: name 'tp53_aligned' is not defined